In [1]:
from reggie.ingestion.download import (
    Preprocessor,
    date_from_str,
    FileItem,
    concat_and_delete,
)
from dateutil import parser
from reggie.ingestion.utils import MissingNumColumnsError, format_column_name
import logging
import pandas as pd
from reggie.configs.configs import Config
import numpy as np
from datetime import datetime
from io import StringIO, BytesIO, SEEK_END, SEEK_SET
import numpy as np
from datetime import datetime
import gc
import json
import os
import math
%load_ext line_profiler


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [2]:
config = Config(state='oklahoma')

In [3]:
voter_files = os.listdir('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/temp')
hist_files = os.listdir('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VH_20210308110646')

In [4]:
voter_files = [n for n in voter_files if "vr.csv" in n.lower()]
hist_files = [n for n in hist_files if "vh.csv" in n.lower()]

In [5]:
vdf = pd.DataFrame()
dtypes = config['dtypes']
cty_map = config['county_codes']

In [18]:
def county_map(pct):
    def mapping(prec):
        county = cty_map[prec[:2]]
        # constant time yo
        print(cty_map['Adair'])
#         county = cty_map[cty_num]
        print(prec)
        print(cty_num)
#         print(prec, type(prec), prec[:2])
        print(county)
#         return county
    return pd.Series(
        map(mapping, pct.tolist())
    )
# why use strings when ints are better
# .values.astype(str)

In [22]:
for file in voter_files:
    print(file)
    temp_df = pd.read_csv('/home/tommi/Downloads/OK 2021-03-06/OK 2021-03-06/CTYSW_VR_20210308110648/' + file, encoding='latin', dtype=dtypes)
    print(temp_df['Precinct'])
    print(temp_df)
    temp_df['county'] = county_map(temp_df['Precinct'])
    vdf = pd.concat([vdf, temp_df], ignore_index=True)

CTY03_vr.csv
0       30011
1       30001
2       30008
3       30011
4       30007
        ...  
7733    30006
7734    30006
7735    30011
7736    30015
7737    30009
Name: Precinct, Length: 7738, dtype: int64
      Precinct    LastName  FirstName     MiddleName Suffix    VoterID  \
0        30011     GOODSON  JOHNATHAN          BRENT   <NA>  800823829   
1        30001     CASSIDY     TERESA           JEAN   <NA>  802091498   
2        30008  INABINETTE   VICTORIA            REA   <NA>  802091782   
3        30011     JENKINS     AUSTIN  TYLER EUBANKS   <NA>  801541030   
4        30007       BATES      JAMES         EDWARD     JR  801541039   
...        ...         ...        ...            ...    ...        ...   
7733     30006   MCFEETERS     VIVIAN          ADELL   <NA>  802853950   
7734     30006        BUSS    RICHARD         EUGENE   <NA>  802847439   
7735     30011      HARVEY    SPENCER         MARLOW   <NA>  802847442   
7736     30015   STARLEIGH     RONALD           LY

NameError: name 'county' is not defined